# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Lab 4</font>
## <font color='blue'>Convertendo Áudio em Texto com Inteligência Artificial</font>

![DSA](imagens/Lab4.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.9


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
!pip install -q transformers==4.30.2

In [4]:
!pip install -q torch==2.0.1

https://pypi.org/project/soundfile/

In [5]:
!pip install -q soundfile

https://pypi.org/project/librosa/

In [6]:
!pip install -q librosa

https://pypi.org/project/datasets/

In [7]:
!pip install -q datasets

In [8]:
# Imports
import torch
import librosa
import transformers
from datasets import load_dataset
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

transformers: 4.30.2
librosa     : 0.10.0.post2
torch       : 2.0.1



## Modelo Facebook Wav2Vec2

Este é um modelo básico pré-treinado e ajustado em 960 horas de Librispeech em áudio de fala amostrado de 16kHz. Ao usar o modelo, certifique-se de que sua entrada de fala também seja amostrada em 16Khz.

https://arxiv.org/abs/2006.11477

https://huggingface.co/facebook/wav2vec2-base-960h

In [10]:
# Carrega o tokenizer e o modelo
tokenizador_1 = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
modelo_1 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


https://huggingface.co/datasets/patrickvonplaten/librispeech_asr_dummy/tree/main

In [11]:
# Carrega conjunto de dados fictício de arquivos de áudio
dados = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split = "validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset librispeech_asr_dummy downloaded and prepared to /Users/dmpm/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc. Subsequent calls will reuse this data.


In [12]:
dados

Dataset({
    features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
    num_rows: 73
})

In [13]:
dados['file']

['/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0000.flac',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0001.flac',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0002.flac',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0003.flac',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0004.flac',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clea

In [14]:
dados['audio']

[{'path': '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0000.flac',
  'array': array([-0.00048828, -0.00018311, -0.00137329, ...,  0.00079346,
          0.00091553,  0.00085449]),
  'sampling_rate': 16000},
 {'path': '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0001.flac',
  'array': array([ 2.74658203e-04,  0.00000000e+00, -3.05175781e-05, ...,
         -3.75366211e-03, -2.07519531e-03, -1.09863281e-03]),
  'sampling_rate': 16000},
 {'path': '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/1b49208c2ca70a81a8b6b265e42d15133fa50de251e9c3a8e30d50bc3f87dfe4/dev_clean/1272/141231/1272-141231-0002.flac',
  'array': array([-4.88281250e-04, -4.27246094e-04, -5.49316406e-04, ...,
         -1.22070312e-04, -9.15527344e-05,  1.22070312e-04]),
  'sampling_rate

In [15]:
# Aplica o tokenizador em 1 arquivo de áudio
input_values = tokenizador_1(dados[0]["audio"]["array"], 
                             return_tensors = "pt", 
                             padding = "longest", 
                             sampling_rate = 16_000).input_values 

In [16]:
input_values

tensor([[-0.0048, -0.0018, -0.0135,  ...,  0.0077,  0.0089,  0.0083]])

In [17]:
# Retorna os logits (previsões do modelo)
logits = modelo_1(input_values).logits

In [18]:
# Argmax com os maiores logits para extrair as previsões
predicted_ids = torch.argmax(logits, dim = -1)

In [19]:
# Previsão de texto do arquivo de áudio
predicted_ids

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  4,  4,  0, 17,  0,
          0,  0,  0,  7,  0,  0,  9,  0,  0,  0,  4,  4,  4,  0,  0,  0,  0, 12,
          0,  0,  0,  7, 10,  0, 14, 14,  4,  4,  6,  6,  0,  8,  0,  4,  4,  6,
         11,  5,  5,  4,  4,  4,  0, 16, 16,  0,  9,  9,  0, 10,  0,  0, 25,  0,
          0,  0,  5, 13, 13,  0,  0,  0, 12,  5,  5,  4,  4,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 12, 12,  0,
          0,  0,  0, 10,  0, 13, 13,  0,  0,  4,  4,  4,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0,  0,  0,  4,
          0,  0,  0,  5, 28,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,  0, 12,
          0,  0,  6,  0,  0,  4,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,

In [20]:
# Tokeniza a previsão
transcription = tokenizador_1.batch_decode(predicted_ids)

In [21]:
transcription

['A MAN SAID TO THE UNIVERSE SIR I EXIST']

## Trabalhando com Áudio em Português

https://huggingface.co/jonatasgrosman/wav2vec2-large-xlsr-53-portuguese

In [22]:
# Modelo
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-portuguese"

In [23]:
# Carrega o tokenizer e o modelo
tokenizer_2 = Wav2Vec2Processor.from_pretrained(MODEL_ID)
modelo_2 = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

In [24]:
# Idioma dos áudios
LANG_ID = "pt"

In [25]:
# Número de amostras
SAMPLES = 10

In [26]:
# Carrega os arquivos de áudio em português do dataset common_voice
dados_teste = load_dataset("common_voice", LANG_ID, split = f"test[:{SAMPLES}]")

Generating train split:   0%|          | 0/6514 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4641 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4592 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/8390 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/41584 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/1740 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /Users/dmpm/.cache/huggingface/datasets/common_voice/pt/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


In [27]:
dados_teste

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10
})

In [28]:
dados_teste['path']

['/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/384437d9e8b301c03eba84b959f87c7b5f9ec58c956a60fb40a9b96ea8a28dfd/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_19273358.mp3',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/384437d9e8b301c03eba84b959f87c7b5f9ec58c956a60fb40a9b96ea8a28dfd/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_19273359.mp3',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/384437d9e8b301c03eba84b959f87c7b5f9ec58c956a60fb40a9b96ea8a28dfd/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_19273360.mp3',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/384437d9e8b301c03eba84b959f87c7b5f9ec58c956a60fb40a9b96ea8a28dfd/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_19273362.mp3',
 '/Users/dmpm/.cache/huggingface/datasets/downloads/extracted/384437d9e8b301c03eba84b959f87c7b5f9ec58c956a60fb40a9b96ea8a28dfd/cv-corpus-6.1-2020-12-11/pt/clips/common_voice_pt_19275111.mp3',
 '/Users/dmpm/.cache/huggingface/dataset

In [29]:
# Função que processa os arquivos de áudio e carregando como arrays
def processa_audio(batch):
    
    # Extrai array e sampling_rate dos arquivos de áudio
    speech_array, sampling_rate = librosa.load(batch["path"], sr = 16_000)
    
    # Array do áudio
    batch["speech"] = speech_array
    
    # Texto (label)
    batch["sentence"] = batch["sentence"].upper()
    
    return batch

In [30]:
# Aplica a função
dados_teste = dados_teste.map(processa_audio)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [31]:
dados_teste

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'speech'],
    num_rows: 10
})

In [32]:
dados_teste["speech"]

[[3.637978807091713e-12,
  -6.366462912410498e-12,
  1.8189894035458565e-12,
  3.637978807091713e-12,
  1.4551915228366852e-11,
  -1.0913936421275139e-11,
  1.0913936421275139e-11,
  -3.637978807091713e-12,
  1.8189894035458565e-11,
  0.0,
  -2.7284841053187847e-12,
  2.546585164964199e-11,
  -2.3646862246096134e-11,
  -1.8189894035458565e-11,
  7.275957614183426e-12,
  1.0913936421275139e-11,
  -3.637978807091713e-12,
  -2.1827872842550278e-11,
  9.094947017729282e-13,
  -2.4556356947869062e-11,
  1.546140993013978e-11,
  -1.546140993013978e-11,
  2.1827872842550278e-11,
  -9.094947017729282e-12,
  -3.637978807091713e-12,
  -2.2737367544323206e-12,
  -1.0913936421275139e-11,
  -1.8189894035458565e-12,
  -1.4551915228366852e-11,
  2.1827872842550278e-11,
  -3.637978807091713e-12,
  2.9103830456733704e-11,
  7.275957614183426e-12,
  -3.637978807091713e-12,
  -1.8189894035458565e-12,
  1.864464138634503e-11,
  0.0,
  3.637978807091713e-12,
  -7.275957614183426e-12,
  -1.4551915228366852e

In [33]:
# Aplica o tokenizador
inputs = tokenizer_2(dados_teste["speech"], return_tensors = "pt", padding = True, sampling_rate = 16_000)

In [34]:
# Extrai os logits (previsões do modelo)
with torch.no_grad():
    logits = modelo_2(inputs.input_values, attention_mask = inputs.attention_mask).logits

In [35]:
# Previsões
predicted_ids = torch.argmax(logits, dim=-1)

In [36]:
# Tokenizer das previsões
predicted_sentences = tokenizer_2.batch_decode(predicted_ids)

In [37]:
# Imprime valor real e valor previsto pelo modelo
for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Valor Real:", dados_teste[i]["sentence"])
    print("Previsão (Texto Extraído do Áudio):", predicted_sentence)

----------------------------------------------------------------------------------------------------
Valor Real: ESTAMOS AQUI PARA PEDIR EMPRESTADO
Previsão (Texto Extraído do Áudio): estamos aqui para pedir emprestado
----------------------------------------------------------------------------------------------------
Valor Real: GRAÇAS A DEUS VOCÊ ESTÁ AQUI!
Previsão (Texto Extraído do Áudio): graças a deus você está aqui
----------------------------------------------------------------------------------------------------
Valor Real: PRECISAMOS NOS APRESSAR.
Previsão (Texto Extraído do Áudio): precisamos nos apressar
----------------------------------------------------------------------------------------------------
Valor Real: A NECESSIDADE É O PAI NA INOVAÇÃO.
Previsão (Texto Extraído do Áudio): a necessidade é pai na inovação
----------------------------------------------------------------------------------------------------
Valor Real: VOCÊ PODERIA TER MORRIDO DEPOIS QUE A PAZ FOSS

# Fim